In [1]:
import datetime
import warnings

import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import skorch

from dataset import StockDataset
from denoising.autoencoder import *

from yahoo_fin_api import load_df

%matplotlib inline
warnings.filterwarnings(action='ignore')

In [7]:
NOW_STR = '28022023-024108'
CONV1_KERNEL = 51
CONV2_KERNEL = 51
CONV3_KERNEL = 51

cnn_encoder = CnnEncoder(CONV1_KERNEL, CONV2_KERNEL, CONV3_KERNEL)
cnn_decoder = CnnDecoder(CONV1_KERNEL, CONV2_KERNEL, CONV3_KERNEL)

cnn_encoder.load_state_dict(torch.load(f'./model-dir/{NOW_STR}-cnn-encoder.pt')), cnn_decoder.load_state_dict(torch.load(f'./model-dir/{NOW_STR}-cnn-decoder.pt'))

(<All keys matched successfully>, <All keys matched successfully>)

In [8]:
cnn_encoder.eval(), cnn_decoder.eval()

(CnnEncoder(
   (encoder_cnn): Sequential(
     (0): Conv1d(1, 8, kernel_size=(51,), stride=(1,), padding=(51,))
     (1): ReLU(inplace=True)
     (2): Conv1d(8, 16, kernel_size=(51,), stride=(1,), padding=(51,))
     (3): ReLU(inplace=True)
     (4): Conv1d(16, 32, kernel_size=(51,), stride=(1,), padding=(51,))
     (5): ReLU(inplace=True)
     (6): Conv1d(32, 64, kernel_size=(51,), stride=(1,), padding=(51,))
     (7): ReLU(inplace=True)
   )
 ),
 CnnDecoder(
   (decoder_cnn): Sequential(
     (0): ConvTranspose1d(64, 32, kernel_size=(51,), stride=(1,), padding=(51,))
     (1): ReLU(inplace=True)
     (2): ConvTranspose1d(32, 16, kernel_size=(51,), stride=(1,), padding=(51,))
     (3): ReLU(inplace=True)
     (4): ConvTranspose1d(16, 8, kernel_size=(51,), stride=(1,), padding=(51,))
     (5): ReLU(inplace=True)
     (6): ConvTranspose1d(8, 1, kernel_size=(51,), stride=(1,), padding=(51,))
   )
 ))

In [ ]:
data = torch.tensor(load_df('data/krw-usd.csv')['Mean'])
# Although the distribution of currency exchange would be very different with stock data, I used the currency exchange data to check whether the autoencoder works properly to other similar data.